In [12]:
import pandas as pd
import matplotlib.pyplot as plt
from  tensorflow.keras.layers import Input,LSTM,Dense,Embedding,TextVectorization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [10]:
data=pd.read_csv("D:\\Source_code\\Lab\\data\\hindi_english_parallel.csv")
eng_text=data['english'].astype('str')
hin_text=data['hindi'].astype('str')

data.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [11]:
# Vectorization layers
max_len = 150
vocab_size = 5000

eng_vectorizer = TextVectorization(max_tokens=vocab_size, output_sequence_length=max_len)
hin_vectorizer = TextVectorization(max_tokens=vocab_size, output_sequence_length=max_len)

eng_vectorizer.adapt(eng_text)
hin_vectorizer.adapt(hin_text)

# Create sequences
eng_seq = eng_vectorizer(eng_text).numpy()
hin_seq = hin_vectorizer(hin_text).numpy()



In [13]:
xtrain,xtest,ytrain,ytest=train_test_split(eng_seq,hin_seq,test_size=0.2,random_state=42)

In [14]:
# ENCODER
encoder_input = Input(shape=(max_len,))
encoder_embed = Embedding(vocab_size, 128)(encoder_input)
_, state_h, state_c = LSTM(128, return_state=True)(encoder_embed)

# DECODER
decoder_input = Input(shape=(max_len - 1,))
decoder_embed = Embedding(vocab_size, 128)(decoder_input)
decoder_lstm = LSTM(128, return_sequences=True)(decoder_embed, initial_state=[state_h, state_c])
decoder_output = Dense(vocab_size, activation='softmax')(decoder_lstm)

# MODEL
model = Model([encoder_input, decoder_input], decoder_output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [20]:
# Train
history = model.fit(
    [xtrain, ytrain[:,:-1]], 
    ytrain[:,1:],
    batch_size=64, 
    epochs=5, 
    validation_split=0.2
)

# Evaluate
loss, acc = model.evaluate([xtest, ytest[:,:-1]], ytest[:,1:])
print(f'Test Accuracy: {acc:.2%}')

Epoch 1/5
    8/15619 ━━━━━━━━━━━━━━━━━━━━ 2:49:22 651ms/step - accuracy: 0.9049 - loss: 0.6736

KeyboardInterrupt: 